In [26]:
import matplotlib.pyplot as plt
import os
import numpy as np
import xarray as xr
import pymap3d as pm
import pandas as pd
import torch

from sklearn.model_selection import train_test_split

# from globenn.model import *

# For reproducibility
os.environ['PYTHONHASHSEED']= '123'
np.random.seed(123)
plt.style.use("../hatsyim.mplstyle")

# Obtaining the 3-D global mantle model

Please refer to the official [GLAD-M25 repository](https://github.com/caiociardelli/gladm25) to obtain the files.

# Data Loading and Preprocessing

In [9]:
path_file = '/home/taufikmh/KAUST/spring_2022/global_pinns/data/glad-m25-vp-0.0-n4.nc'
data = xr.open_dataset(path_file)

offline = False
old = False

# Subsample the model along the depth, latitude, and longitude directions
if offline and old:
    dep_ini, dep_inc = 11, 4
    lat_ini, lat_inc = 0, 8
    lon_ini, lon_inc = 0, 8
else:
    dep_ini, dep_inc = 11, 2
    lat_ini, lat_inc = 0, 4
    lon_ini, lon_inc = 0, 4    

# Input variables
vpv = data.variables['vpv'].values[dep_ini::dep_inc, lat_ini::lat_inc, lon_ini::lon_inc]
vph = data.variables['vph'].values[dep_ini::dep_inc, lat_ini::lat_inc, lon_ini::lon_inc]
longitude = data.variables['longitude'].values[lon_ini::lon_inc]
latitude = data.variables['latitude'].values[lat_ini::lat_inc]
depth = data.variables['depth'].values[dep_ini::dep_inc]

dep_dim = vpv.shape[0]
lat_dim = vpv.shape[1]
lon_dim = vpv.shape[2]

# Projection
LAT, ALT, LON = np.meshgrid(latitude, -1e3*depth, longitude)
x, y, z = pm.geodetic2ecef(LAT, LON, ALT)
_, DEP, _ = np.meshgrid(latitude, depth, longitude)

# Medium and Saving parameters

In [13]:
# Load all stations
ISCall = pd.read_csv('../data/stations.csv')
ISCall = ISCall.rename(columns={"X":"LON", 'Y':'LAT'})
ISCarray = ISCall[ISCall['description'].str.contains('to 2021')]
ISCarrLon = ISCarray['LON'][::5]
ISCarrLat = ISCarray['LAT'][::5]

USarray = pd.read_excel('../data/_US-TA-StationList.xls')
USarrLon = USarray['LON'][::1]
USarrLat = USarray['LAT'][::1]

# Concatenate the two receiver group
AllLon = np.hstack((USarrLon, ISCarrLon))
AllLat = np.hstack((USarrLat, ISCarrLat))

# Model specifications
ear_rad = 6371
rec_typ = 'US_array'
if rec_typ == 'ISC_array':
    print(rec_typ)
    lat_sou = np.array([latitude.flat[np.abs(latitude - i).argmin()] for i in ISCarrLat])
    lon_sou = np.array([longitude.flat[np.abs(longitude - i).argmin()] for i in ISCarrLon])
elif rec_typ == 'US_array':
    print(rec_typ)
    lat_sou = np.array([latitude.flat[np.abs(latitude - i).argmin()] for i in USarrLat])
    lon_sou = np.array([longitude.flat[np.abs(longitude - i).argmin()] for i in USarrLon])
dep_sou = depth.flat[np.abs(depth - 0).argmin()]

# Plotting coordinates
xx = (ear_rad - DEP) * np.sin(np.radians(LAT+90)) * np.cos(np.radians(180+LON))/(1e3)
yy = (ear_rad - DEP) * np.sin(np.radians(LAT+90)) * np.sin(np.radians(180+LON))/(1e3)
zz = DEP * np.cos(np.radians(LAT+90)) / (1e3)
xx_s = (ear_rad - dep_sou) * np.sin(np.radians(lat_sou+90)) * np.cos(np.radians(180+lon_sou))/(1e3)
yy_s = (ear_rad - dep_sou) * np.sin(np.radians(lat_sou+90)) * np.sin(np.radians(180+lon_sou))/(1e3)

# Coordinates setup
sx, sy, sz = pm.geodetic2ecef(lat_sou, lon_sou, -1e3*dep_sou)

# Rescale
x,y,z = x/(ear_rad*1e3), y/(ear_rad*1e3), z/(ear_rad*1e3)
sx, sy, sz = sx/(ear_rad*1e3), sy/(ear_rad*1e3), sz/(ear_rad*1e3)
X,Y,Z = x,y,z

sou_idx = np.where((np.isclose(x.reshape(-1,1), sx)) & (np.isclose(y.reshape(-1,1), sy)) & (np.isclose(z.reshape(-1,1), sz)))[0]
sx, sy, sz = x.reshape(-1,1)[sou_idx], y.reshape(-1,1)[sou_idx], z.reshape(-1,1)[sou_idx]

# For plotting only
x_plot,y_plot,z_plot = x.reshape(-1,1)*ear_rad/1000, y.reshape(-1,1)*ear_rad/1000, z.reshape(-1,1)*ear_rad/1000
num_pts = x.size

PROJECT_NAME='glad-m25'

# Saving path
model_path = "./../saves/" + PROJECT_NAME
figures_path = model_path + '/'
checkpoints_path = figures_path + 'checkpoints' + '/'
predictions_path = figures_path + 'predictions' + '/'

from pathlib import Path

Path(figures_path).mkdir(parents=True, exist_ok=True)
Path(checkpoints_path).mkdir(parents=True, exist_ok=True)
Path(predictions_path).mkdir(parents=True, exist_ok=True)

In [24]:
try_first = False
single_source = True
permutate = False

num_epo = int(2001)
num_blo = 21 #20
coo_sys = 'cartesian'
vel_sha = 'sphere'
vel_typ = 'gladm25'
num_neu = 512
lea_rat = 5e-6
act_fun = torch.nn.ELU
bat_siz = 512 #num_pts // 100
ada_wei = False
vel_sca = 1
opt_fun = torch.optim.Adam
dev_typ = "cuda"
nor_typ = "MinMax"
bac_vel = 10 #6 #10.5

In [21]:
def velocity_function(vel_sha='sphere', vel_typ='homogeneous', vel_ini=5, x=X, y=Y, z=Z):
    if vel_sha=='sphere':
        # inhomogeneous velocity without gaps
        vel_inh = vel_ini - (x**2 + z**2 + y**2)
        vel_inh[vel_inh<2.**2] = np.NaN
        if vel_typ=='inhomogeneous':  
            # velocity without gaps
            vel_all = np.copy(vel_inh)
            
            # velocity with gaps
            vel_gap = np.copy(vel_all)
            vel_gap[vel_inh>2.1**2] = np.NaN
        elif vel_typ == 'random':
            # velocity without gaps
            from scipy.ndimage import gaussian_filter
            vel_all = gaussian_filter(18*np.random.random((nx,ny,nz))**3, sigma=6)
            vel_all[np.isnan(vel_inh)] = np.NaN

            # velocity with gaps
            vel_gap = np.copy(vel_all)
            vel_gap[vel_inh>2.1**2] = np.NaN
        elif vel_typ == 'homogeneous':
            # velocity without gaps
            vel_all = vel_ini*np.ones_like(vel_inh)
            vel_all[np.isnan(vel_inh)] = np.NaN
            
            # velocity with gaps
            vel_tmp = np.copy(vel_inh)
            vel_tmp[vel_inh>2.1**2] = np.NaN
            vel_gap = np.copy(vel_all)
            vel_gap[np.isnan(vel_tmp)] = np.NaN
        elif vel_typ == 'radial':
            vel_inh = vp
            vel_all = vp
            vel_gap = vp_gap
        elif vel_typ == 'constant_radial':
            vel_inh = 5*np.ones_like(vp)
            vel_all = 5*np.ones_like(vp)
            vel_gap = np.where(DEP<3000, vel_all, np.nan)
        elif vel_typ == 'gladm25':
            vel_inh = vpv
            vel_all = vpv
            vel_gap = vpv
            
            
    return vel_inh, vel_all, vel_gap

# call function
vel_inh, vel_all, vel_gap = velocity_function(vel_typ=vel_typ)

# Training Points

In [27]:
# define receiver coordinates
xR, yR, zR = X.reshape(-1,1), Y.reshape(-1,1), Z.reshape(-1,1)

# define source coordinates
if single_source:
    xS, yS, zS = sx[0]*np.ones_like(X.reshape(-1,1)), sy[0]*np.ones_like(X.reshape(-1,1)), sz[0]*np.ones_like(X.reshape(-1,1))
else:
    xS, yS, zS = np.tile(sx,(X.size//sx.shape[0]+1, 1))[:X.size].reshape(-1,1), np.tile(sy,(Y.size//sy.shape[0]+1,1))[:Y.size].reshape(-1,1), np.tile(sz,(Z.size//sz.shape[0]+1,1))[:Z.size].reshape(-1,1)

# define inputs and output
Xp = np.hstack((xS, yS, zS, xR, yR, zR))
yp = vel_gap.reshape(-1,1)

# input for database
Xb = np.copy(Xp)
yb = np.copy(yp)

print(Xb.size, lat_dim*dep_dim*lon_dim)

if permutate:
    perm_idx = torch.randperm(X.size).numpy()
    Xb[:, :3] = Xb[perm_idx, :3]

# random sampling
X_train, X_test, y_train, y_test = train_test_split(
    Xp[np.logical_not(np.isnan(yp))[:,0]], 
    yp[np.logical_not(np.isnan(yp))].reshape(-1,1), 
    test_size=0.1,
    random_state=1335
)

# find source location id in X_star
X_starf = [X_train[:,3].reshape(-1,1), X_train[:,4].reshape(-1,1), X_train[:,5].reshape(-1,1)]

# sids,_ = np.where((np.isclose(X_starf[0], sx)) & (np.isclose(X_starf[1], sy)) & (np.isclose(X_starf[2], sz)))

sids = [np.where((np.isclose(X_starf[0], sx[i], atol=1e-16)) & (np.isclose(X_starf[1], sy[i], atol=1e-16)) & (np.isclose(X_starf[2], sz[i], atol=1e-16))) for i in range(len(sx))]
sids = np.asarray(sids)[:,0]

print(sids.shape)

sids = sids[sids.astype(bool)].astype(int)

print("after")
print(sids)
print(X_starf[0][sids,0])
print(X_starf[1][sids,0])
print(X_starf[2][sids,0])
print(sx,sy,sz)

16405116 2734186
(2234,)
after
[1309567  847284  847284 ...  248151  248151 1837617]
[ 0.12723418 -0.12519088 -0.12519088 ... -0.28321096 -0.28321096
 -0.18222106]
[-0.90531814 -0.8907796  -0.8907796  ... -0.12609363 -0.12609363
 -0.25080577]
[0.40430745 0.43579215 0.43579215 ... 0.94772875 0.94772875 0.94772875]
[[ 0.12723418]
 [-0.12519088]
 [-0.12519088]
 ...
 [-0.28321096]
 [-0.28321096]
 [-0.18222106]] [[-0.90531814]
 [-0.8907796 ]
 [-0.8907796 ]
 ...
 [-0.12609363]
 [-0.12609363]
 [-0.25080577]] [[0.40430745]
 [0.43579215]
 [0.43579215]
 ...
 [0.94772875]
 [0.94772875]
 [0.94772875]]


/tmp/ipykernel_2817/618570644.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sids = np.asarray(sids)[:,0]
/tmp/ipykernel_2817/618570644.py:42: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  sids = sids[sids.astype(bool)].astype(int)


In [34]:
# from globenn.utils import *
database = database(model_path,
         VelocityFunction(),
         create=False,
         Numsamples=int(1e3),
         randomDist=False
        )

0.0


In [35]:
num_epo = int(2001)
num_blo = 21 #20
coo_sys = 'cartesian'
vel_sha = 'sphere'
vel_typ = 'gladm25'
num_neu = 512
lea_rat = 5e-6
act_fun = torch.nn.ELU
bat_siz = 128 #num_pts // 100
ada_wei = False
vel_sca = 1
opt_fun = torch.optim.Adam
dev_typ = "cuda"
nor_typ = "MinMax"
bac_vel = 10 #6 #10.5

offline, old = False, False

# Model parameters
params = {}
params['model_path'] = model_path
params['device'] = dev_typ
params['mixed_precision'] = False
params['num_blocks'] = num_blo
params['act_function'] = act_fun
params['num_points'] = 1e4
params['batch_size'] = bat_siz
params['val_percentage'] = 10
params['num_epochs'] = num_epo
params['sampling_bounds'] = [0.1,0.9]
params['log_frequency'] = 1
params['save_frequency'] = 10
params['learning_rate'] = lea_rat
params['use_scheduler'] = True

PROJECT_NAME='new-section-1'

# Saving path
model_path = "./../saves/" + PROJECT_NAME
figures_path = model_path + '/'
checkpoints_path = figures_path + 'checkpoints' + '/'
predictions_path = figures_path + 'predictions' + '/'

from pathlib import Path

Path(figures_path).mkdir(parents=True, exist_ok=True)
Path(checkpoints_path).mkdir(parents=True, exist_ok=True)
Path(predictions_path).mkdir(parents=True, exist_ok=True)

# Train

In [39]:
# from globenn.model import *

model = Model(params)

model.train()

cuda:0
Epoch = 1 -- Training loss = 1.4412e+02 -- Validation loss = 3.5440e+01


KeyboardInterrupt: 

# Test and Save

In [ ]:
Xt = torch.utils.data.DataLoader(
    torch.from_numpy(Xb).to(torch.float).to(torch.device('cuda')),
    batch_size=int(Xb.shape[0]//10)
    )
T_pred = model.traveltimes(Xt, projection=False, normalization=True).cpu().reshape(vel_all.shape)
V_pred = model.velocity(Xt, projection=False, normalization=True).cpu().reshape(vel_all.shape)
    
T_pred, V_pred = T_pred.detach().numpy(), V_pred.detach().numpy()

# numpy to hdf5
predictions = [
    T_pred,
    V_pred
    
]
name = [
    'T_pred',
    'V_pred'
]
print(np.min(T_pred) , np.min(T_pred)==0)
for i in range(len(predictions)):
    h5f = h5py.File(predictions_path + name[i]+'.h5', 'w')
    h5f.create_dataset(name[i], data=predictions[i])
    h5f.close()